In [1]:
import requests
from bs4 import BeautifulSoup

from pymongo import MongoClient           # pymongo를 임포트 하기(패키지 인스톨 먼저 해야겠죠?)
my_client = MongoClient('localhost', 27017)  # mongoDB는 27017 포트로 돌아갑니다.
my_db = my_client.smkim                      # 'smkim'라는 이름의 db를 만듭니다.
movie_collections = my_db.movies

# URL을 읽어서 HTML를 받아오고,
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}
data = requests.get('https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date=20200303',headers=headers)

# HTML을 BeautifulSoup이라는 라이브러리를 활용해 검색하기 용이한 상태로 만듦
soup = BeautifulSoup(data.text, 'html.parser')

# select를 이용해서, tr들을 불러오기
movies = soup.select('#old_content > table > tbody > tr')

# movies (tr들) 의 반복문을 돌리기
for movie in movies:
    # movie 안에 a 가 있으면,
    a_tag = movie.select_one('td.title > div > a')
    if a_tag is not None:
        rank = movie.select_one('td:nth-of-type(1) > img')['alt'] # img 태그의 alt 속성값을 가져오기
        title = a_tag.text                                      # a 태그 사이의 텍스트를 가져오기
        star = movie.select_one('td.point').text                # td 태그 사이의 텍스트를 가져오기
        doc = {
            'rank' : rank,
            'title' : title,
            'star' : star
        }
        movie_collections.insert_one(doc)

In [11]:
### 저장 결과 확인
saved_value = list(movie_collections.find({}, {'_id':False}))
print('저장 결과 확인 : ', len(saved_value), '\n', saved_value[:3])

저장 결과 확인 :  50 
 [{'rank': '01', 'title': '그린 북', 'star': '9.60'}, {'rank': '02', 'title': '가버나움', 'star': '9.59'}, {'rank': '03', 'title': '베일리 어게인', 'star': '9.52'}]
